In [55]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax, expit

In [56]:
from loaders import clean_text_for_pattern_letter, get_pattern_for_letter_without_collision
from temp_text import template_str
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
print(len(input_str))

letter_pattern = get_pattern_for_letter_without_collision(size=128, pattern_size=4, dump_cache=True)
print(len(letter_pattern))

len(letters)=32
500
a 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
b 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
c 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,

In [57]:
samples_l = []

for inp in input_str:
    pattern = letter_pattern[inp]
    samples_l.append(pattern)

samples = np.array(samples_l)
print(samples.shape)

(500, 128)


In [58]:
def swish(x):
    return x * expit(x)

In [59]:
def normalize(x):
    
    mean = x.mean(axis = 0)
    std = x.std(axis = 0)
    
    return (x - mean) / np.sqrt(std + 1e-5)

In [60]:
def softmax_grad(err_grad):
    # Reshape the 1-d softmax to 2-d so that np.dot will do the matrix multiplication
    s = err_grad.reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)

In [61]:
def z_fd(z, err_grad):
    sfm = np.reshape(z, (1, -1))
    grad = np.reshape(err_grad, (1, -1))

    d_softmax = (sfm * np.identity(sfm.size) - sfm.transpose() @ sfm)

    return (grad @ d_softmax).ravel()

In [62]:
def show_inp_mat(M):
    print(M.shape)
    plt.matshow(M.T)
    plt.colorbar()
    plt.show()

In [63]:
# train
col_size = 32

samples_size = 10
samples = samples[:samples_size]

mu = 0.0
sigma = 1.0

I = np.zeros((samples.shape[1], col_size))
O = np.zeros(I.shape)
W = np.random.normal(mu, sigma, (I.shape[0], I.shape[1], I.size))
E = np.zeros(I.shape[0])

dW = np.zeros(W.shape)

af_pow = 2

I = np.random.rand(I.shape[0], I.shape[1]) * 1e-3
I = np.power(I, af_pow)
# I = softmax(I)
# I = normalize(I)

epoch_cnt = 100
eps0 = 0.02

for epoch_idx in range(epoch_cnt):
    eps = (epoch_cnt - epoch_idx) * eps0

    batch_E = []

    print('W=',W.shape, W.max(), W.min(), np.mean(W))

    for i_idx in range(samples.shape[0] - 1):
    # for i_idx in range(1):
        next_idx = i_idx + 1

        IV = samples[i_idx]
        OVR = samples[next_idx]

        # infer
        I = I * IV[np.newaxis].T

        # show_inp_mat(I)

        O = np.tensordot(W, I.reshape(I.size), 1)

        # show_inp_mat(O)

        O = np.power(O, af_pow)
        # O = softmax(O)
        # O = normalize(O)
        # O /= np.linalg.norm(O, ord=2)

        # show_inp_mat(O)

        OV = np.sum(O, axis=1)
        # show_inp_mat(OV[np.newaxis].T)
        # TODO: norm OV and norm OV derivative?!

        E = (OVR - OV) ** 2
        # show_inp_mat(E[np.newaxis].T)
        print(E.sum())
        batch_E.append(E.sum())

        # bp
        EO = E[np.newaxis].T * np.ones(O.shape[1])
        # show_inp_mat(EO)

        # EO = softmax_grad(EO)
        # EO = z_fd(O, EO).reshape(O.shape)
        # show_inp_mat(EO)

        EO = af_pow * np.power(EO, af_pow-1)
        # show_inp_mat(EO)

        dW += EO.reshape(EO.shape[0], EO.shape[1], 1) * I.reshape(I.size).reshape(1, 1, I.size)
        # print(EO.reshape(EO.shape[0], EO.shape[1], 1)[0][0])
        # print(I.reshape(I.size).reshape(1, 1, I.size)[0][0].size)
        print(dW.shape, dW.max(), dW.min())

        # O norm
        O /= np.linalg.norm(O, ord=1)
        # O /= np.max(np.abs(O))
        # O = softmax(O)

        # reccurention
        I = np.copy(O)

    # update
    print('W(before_update)=',W.shape, W.max(), W.min(), np.mean(W))
    W -= dW * eps

    err = np.mean(np.array(batch_E))
    print(f'{err=}')
    print('W(updated)=',W.shape, W.max(), W.min(), np.mean(W))


W= (128, 32, 4096) 5.6218826114932074 -5.411302992863804 0.00012509350245497414
3.9999999942071556
(128, 32, 4096) 1.9859577024572176e-06 0.0
23.06116563336362
(128, 32, 4096) 0.05074810276518961 0.0
23.436182361120792
(128, 32, 4096) 0.06597911958679284 0.0
45.05167910978055
(128, 32, 4096) 0.08043368470750244 0.0
31.478508698655034
(128, 32, 4096) 0.10418487552194874 0.0
43.381732488376784
(128, 32, 4096) 0.10794614435675323 0.0
63.61901310382256
(128, 32, 4096) 0.17745513748466776 0.0
17.297813712156405
(128, 32, 4096) 0.17745513748466776 0.0
13.111810969707923
(128, 32, 4096) 0.17745513748466776 0.0
W(before_update)= (128, 32, 4096) 5.6218826114932074 -5.411302992863804 0.00012509350245497414
err=29.381989563465645
W(updated)= (128, 32, 4096) 5.6218826114932074 -5.411302992863804 -0.00149128772495137
W= (128, 32, 4096) 5.6218826114932074 -5.411302992863804 -0.00149128772495137
9.645563725528454
(128, 32, 4096) 0.17745513748466776 0.0
12.511355540749623
(128, 32, 4096) 0.17745513748

C:\Users\Acer\AppData\Local\Temp\ipykernel_3028\1214161649.py:50: RuntimeWarning: overflow encountered in power
  O = np.power(O, af_pow)
C:\Users\Acer\AppData\Local\Temp\ipykernel_3028\1214161649.py:61: RuntimeWarning: overflow encountered in square
  E = (OVR - OV) ** 2
C:\Users\Acer\AppData\Local\Temp\ipykernel_3028\1214161649.py:77: RuntimeWarning: invalid value encountered in multiply
  dW += EO.reshape(EO.shape[0], EO.shape[1], 1) * I.reshape(I.size).reshape(1, 1, I.size)


(128, 32, 4096) nan nan
nan


C:\Users\Acer\AppData\Local\Temp\ipykernel_3028\1214161649.py:83: RuntimeWarning: invalid value encountered in divide
  O /= np.linalg.norm(O, ord=1)


(128, 32, 4096) nan nan
nan
(128, 32, 4096) nan nan
nan
(128, 32, 4096) nan nan
nan
(128, 32, 4096) nan nan
nan
(128, 32, 4096) nan nan
nan
(128, 32, 4096) nan nan
nan
(128, 32, 4096) nan nan
W(before_update)= (128, 32, 4096) 5.6218826114932074 -3.1663017560794716e+299 -2.569192310149587e+295
err=nan
W(updated)= (128, 32, 4096) nan nan nan
W= (128, 32, 4096) nan nan nan
nan
(128, 32, 4096) nan nan
nan
(128, 32, 4096) nan nan
nan


KeyboardInterrupt: 